## 🐦🎵 Processing Chirped Pulses: Resolving Overlapping Echoes  

We’ve learned the **physical principle** behind transmitting a **long pulse** to achieve **high energy**, while still maintaining the **resolution of a short pulse**. But how do we actually process the data to **recover this resolution**?  🔍

We can't simply ask the user to **analyze the changing frequencies** in the received signal—like a **professional musician listening for overlapping scales** 🎶—even though that is, in essence, the **physical mechanism** separating the echoes. Instead, we need a **systematic approach** to extract the target information.  

---

## 🕵️ Detecting Fingerprints   

Imagine a **forensic investigator** dusting for **fingerprints** on a table. They have a **reference fingerprint** 🖐️ and need to check if it appears anywhere on the surface.  

1️⃣ **Sliding the Reference Print** ⬅️➡️  
   - The investigator **moves** the fingerprint sample across different areas of the table.  

2️⃣ **Comparing Patterns** 🔎  
   - At each position, they **compare** the reference fingerprint with the dust traces left behind.  

3️⃣ **Finding Strong Matches** ✅  
   - Some areas show a **clear match**, revealing distinct fingerprint ridges.  

4️⃣ **Detecting Partial Matches** 🤔  
   - Other areas may have **partial traces**—still a match, but weaker.  

Instead of identifying just **one exact location**, the investigator creates a **map** 🗺️, showing how likely it is that the fingerprint appears at each position.  
- Areas with a **strong match** 🔴 indicate a **high probability** of the fingerprint being present.  
- Areas with **faint traces or no match** ⚪ suggest low or no correlation.  

💡 **Why is this important?**  
This process is **just like radar signal processing!** Instead of scanning for fingerprints, we **slide a reference signal** across the received data to **detect echoes and measure correlations**—a key concept in **matched filtering**! 📡✨  

---

## 📡 Cross-Correlation in Radar  

**Cross-correlation** is a mathematical operation that follows the same principle as our **fingerprint analogy**! 🕵️✨  

- The **reference fingerprint** 🖐️ is like the **transmitted chirp signal**.  
- The **dust on the table** represents the **received radar signal**, which may contain **overlapping chirp echoes** from multiple targets.  
- As we **slide the transmitted chirp** over the received echoes, we **measure how much of it is present at each time delay**.  
- **Strong matches** correspond to **targets**, while **weaker matches** indicate **noise or no echoes**.  

### 🔎 What Does Cross-Correlation Do?  
Cross-correlation **measures how much of the chirp exists** at different **time delays, i.e., ranges**.

Here's how it works: ⚙️

1. A copy of the **transmitted chirp waveform** (reference signal) is **shifted** step-by-step across different time delays (corresponding to different target ranges). ⏩  
2. At each time-dealy step, the **phase of the shifted reference chirp** is **subtracted** from the **phase of the received echo signal**. ⚡ (This can be done efficiently using **phasor multiplications**! ↗️✖️↘️) If an echo arrived with this delay, all of its samples will now **align in phase**. 🎯  
3. The phase-adjusted values (phasors) are then **summed** to produce a **correlation value** for that particular shift. ➕  
4. This process is **repeated for each delay**, creating an output that highlights where the received echoes **best matches** the transmitted signal. 🔍📶

### ⚡ Why This Matters:
- By transmitting a **long coded pulse** (e.g., with frequency modulation), we maintain **high total energy** for better detection range.  
- After **cross-correlation**, the received signal is **compressed** into a much **shorter, high-resolution peak**, improving **range resolution** while keeping **high energy** in the signal. 🚀  

This technique enables radars to **see further** and **resolve finer details** without violating the power limitations of short pulses! ⚖️📶  


---

## 🛠️ From Cross-Correlation to Resolution: Aligning Phases  

To fully recover high resolution, we need to **refocus** the energy of each chirped pulse. The process of cross-correlation is very similar to what we did with **beam steering**:  

- In **antenna arrays**, signals arrive at different **angles**, creating **phase differences** between receiver elements. By **adjusting the phase shifts** of each element, we can steer the beam and separate overlapping echoes.  
- In **chirp echoes**, signals arrive at different **delays**, creating a mix of frequencies in each range bin. To separate them, we **adjust the phases** of the received signal so that each echo sums coherently at its correct delay—just like focusing a beam, but in **time** instead of space!  

By **coherently summing** the samples of each echo, we can **compress the energy** into sharp peaks at their correct delays—**revealing target positions with high resolution**. 🎯✨  

---

## 🧩 Matched Filter Processing: The Power of Cross-Correlation  

Applying **cross-correlation** in radar is known as **matched filter processing**. Why? Because the **template**—our transmitted chirp—is **perfectly matched** to what we expect in the received signal: a **delayed and attenuated chirp**.  

✅ **What does this achieve?**  
- Maximizes **detection of known signals** 📡  
- Enhances our ability to extract **weak echoes buried in noise** 🔊✨  
- Enables **high-resolution radar imaging** by prducing a **highly concentrated peak** of wave energy 🌊

---

## 🎬 Matched Filter Processing in Action  

The animation below **illustrates matched filter processing** in radar. 📡✨  

🔊 We have **two overlapping chirp echoes**, each returning with a **slightly different time delay**.  
📈 By calculating the **cross-correlation** with the transmitted chirp, we **focus the target energy** into **narrow peaks**, revealing their precise **time delays**. 🎯🚀  

This process allows us to **separate echoes that would otherwise blend together**, enhancing **range resolution** and improving **target detection**! 🔍  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 50  # Increase max animation size to X MB

# Define signals & parameters

# Chirp rate (same for all chirps)
chirp_rate = 2.0 # 1 / s^2

# Define two delays (targets)
delay1 = 0.0 # s
delay2 = 0.5 # s

# Pulse duration and time array
pulse_duration = 4.0
total_time = 8.0

# Let's calculate the require delay spacing using Nyquist. The bandwidth of the chirp is chirp_rate * pulse_duration
nyquist_frequency = 2 * chirp_rate * pulse_duration
time_samples = 2*int( total_time * nyquist_frequency )
t = np.linspace(-total_time/2, total_time/2, time_samples)
dt = t[1] - t[0]

def rect(t, width=1.0, center=0.0):
    return np.where((t >= center - width/2) & (t <= center + width/2), 1.0, 0.0)

def chirp_signal(t, chirp_rate=1.0):
    """
    Returns a chirp signal s(t) = exp(j * pi * chirp_rate * t^2).
    Instantaneous frequency is f(t) = chirp_rate * t.
    """
    return np.exp(1j * np.pi * chirp_rate * t**2)

# Create two chirps: one at delay1, another at delay2
chirp1 = chirp_signal(t - delay1, chirp_rate) * rect(t, pulse_duration, delay1)
chirp2 = chirp_signal(t - delay2, chirp_rate) * rect(t, pulse_duration, delay2)

# Summation => simulated received signal
rx_signal = chirp1 + chirp2

# We will shift the reference chirp to num_frames different delays to calculate the cross-correlation
# Let's oversample by a factor of 2 to show the shape nicely
num_frames = 2 * int( total_time * nyquist_frequency )
shifts = np.linspace(-total_time/2, total_time/2, num_frames)

# Prepare arrays for overlap and correlation
overlap_data = np.zeros((num_frames, len(t)), dtype=complex)
corr_values = np.zeros(num_frames, dtype=complex)

# Precompute overlap & correlation for each shift
for i, shift in enumerate(shifts):
    # Shift the reference chirp by 'shift'
    shifted_ref = chirp_signal(t - shift, chirp_rate)
    
    # Cross-correlation: multiply rx_signal by conj(shifted_ref)
    overlap = rx_signal * np.conjugate(shifted_ref)
    overlap_data[i] = overlap
    
    # Simple sum of overlap
    corr_values[i] = np.sum(overlap)

# Figure for subplots
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(8, 10), sharex=False)
plt.tight_layout(pad=3.0)

# Plot individual chirps
ax0.set_title('Individual Delayed Chirps')
ax0.plot(t, np.real(chirp1), color='blue',  label=f'Chirp 1 (delay={delay1})')
ax0.plot(t, np.real(chirp2), color='red',   label=f'Chirp 2 (delay={delay2})')
ax0.set_xlim(t[0], t[-1])
# Adjust if you want more room
min_val = min(np.real(chirp1).min(), np.real(chirp2).min(), -1.2)
max_val = max(np.real(chirp1).max(), np.real(chirp2).max(), 1.2)
ax0.set_ylim(min_val, max_val)
ax0.legend()
ax0.set_xlabel('Time (s)')
ax0.set_ylabel('Amplitude')

# Received overlapping chirps and sliding template
ax1.set_title('Received Signal (Sum of Delayed Chirps)')
line_rx, = ax1.plot(t, np.real(rx_signal), color='blue', label='Received echo signal')
line_ref, = ax1.plot([], [], 'r--', label='Shifted reference chirp')
# Mark the target delays with dashed vertical lines
ax1.axvline(delay1, color='red', linestyle='--', label=f'Delay 1 = {delay1}')
ax1.axvline(delay2, color='green', linestyle='--', label=f'Delay 2 = {delay2}')
ax1.legend()
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Amplitude')
ax1.set_xlim(t[0], t[-1])

# Plot for correlation magnitude
ax2.set_title('Correlation Output (Magnitude)')
ax2.axvline(delay1, color='red', linestyle='--', label=f'Delay 1 = {delay1}')
ax2.axvline(delay2, color='green', linestyle='--', label=f'Delay 2 = {delay2}')
line_corr, = ax2.plot([], [], 'g', label='|Correlation|')
ax2.legend()
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Magnitude')
ax2.set_xlim(shifts[0], shifts[-1])
max_corr = np.max(np.abs(corr_values))
ax2.set_ylim(0, 1.1 * max_corr)

def animate(i):
    shift = shifts[i]
    
    # Received signal and shifted reference
    shifted_ref = chirp_signal(t - shift, chirp_rate) * rect(t, pulse_duration, shift)
    line_ref.set_data(t, np.real(shifted_ref))
    
    # Correlation magnitude up to this shift
    line_corr.set_data(shifts[:i+1], np.abs(corr_values[:i+1]))
    
    return line_ref, line_corr

chirp_animation = animation.FuncAnimation(
    fig,
    animate,
    frames=num_frames,
    interval=100,  # ms between frames
    blit=True
)

HTML(chirp_animation.to_jshtml())

## ⚡ Range Compression: Sharpening Radar Echoes  

The process of **calculating the correlation** of the received echo signal with the transmitted chirp is called **range compression** in radar lingo. 🗜️ Why? Because it **"compresses"** the energy of a long pulse into a **narrow, high peak**—sharpening the radar’s ability to detect targets with precision. 🎯  

---

### 📏 What Determines Range Resolution?  

The **resolution** is defined by **the width of the compressed peak**. And what determines this width? **Surprise, surprise—it's the bandwidth!** 📻  

- The **wider** the bandwidth (the difference between maximum and minimum frequency), the **narrower** the compressed peak.  
- This means we achieve **the same resolution** that we would have gotten if we had transmitted a **short pulse of the same bandwidth**, but without the chirp modulation!  

---

### 🪄 The Magic of Pulse Compression  

Here’s the real **game-changer**:  By calculating the correlation, we **collect the entire energy of the chirped pulse into a sharp peak**.  

✨ **It’s as if we transmitted an extremely powerful, short pulse!** ✨  

But instead of requiring **massive instantaneous transmit power**, we achieve this entirely **digitally**—by processing the recorded waves in a computer. **That’s real radar magic!** 🔮  

---

### 🔊 Boosting SNR: Why Pulse Compression Helps in Noise  

One of the **coolest** advantages of collecting the **entire pulse energy** into a peak is that it **improves the signal-to-noise ratio (SNR)**.  

Imagine that the **chirp is buried in strong background noise**. What happens when we compute the **cross-correlation**?  

✅ **At the correct target delay**, the wave samples from the entire chirp **align in phase** and sum together **coherently**, reinforcing the signal.  
✅ **The noise, however, is random**—its phases do not align, so it does **not sum coherently** like the signal does.  
✅ As a result, **the signal energy increases**, but the **noise remains mostly unchanged**, effectively **boosting the SNR!** 🎯  

---

### 🛠️ Experiment: Play with Bandwidth & SNR  

The code snippet below lets you **explore** two fundamental radar concepts:  

🔹 **Range Compression & Resolution** 🎯  
   - Increase the **bandwidth** and watch as the **compressed peaks sharpen**, allowing you to **resolve closely spaced targets** with greater ability.  

🔹 **SNR Improvement** 🔊➡️📈  
   - Lower the **SNR** to **bury the echoes in noise**—then see how **cross-correlation still reveals the targets!**  
   - Even when the **raw echo signal disappears into noise**, the **matched filter output** brings the targets **back into focus**! 👀✨  

🚀 **Try it out and see how these factors impact radar performance!** 🔍📡 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# User-defined parameters
bandwidth = 1e6  # Hz - Adjust this and see what happens
SNR_dB = 10  # Signal-to-Noise Ratio in dB - Compare the raw signal with the correlation output. 
# (0 means equal noise and signal power in raw data)

# pulse length
pulse_duration = 2e-5  # seconds

# Speed of light (m/s)
c = 3e8  

# Convert bandwidth to chirp rate
chirp_rate = bandwidth / pulse_duration  # Hz/s
print("Range resolution:", c / (2*bandwidth), "m")

# Define target ranges (meters)
target_range1 = 2000  # meters
target_range2 = 2100  # meters

# Convert range to time delay (round-trip time)
delay1 = 2 * target_range1 / c  # seconds
delay2 = 2 * target_range2 / c  # seconds

# Time array setup
total_time = max(delay1, delay2) + pulse_duration  # Ensure it covers both targets
nyquist_frequency = 2 * bandwidth  # Nyquist limit
time_samples = 2 * int(total_time * nyquist_frequency)  # 2 x oversampling
t = np.linspace(0, total_time, time_samples)
dt = t[1] - t[0]
range_axis = np.linspace(0, c * time_samples * dt / 2, time_samples)

def rect(t, width=1.0, center=0.0):
    """Returns a rectangular window of given width centered at 'center'."""
    return np.where((t >= center - width/2) & (t <= center + width/2), 1.0, 0.0)

def chirp_signal(t, chirp_rate=1.0):
    """Generates a chirp signal s(t) = exp(j * pi * chirp_rate * t^2)."""
    return np.exp(1j * np.pi * chirp_rate * t**2)

# Create two chirps with different delays
chirp1 = chirp_signal(t - delay1, chirp_rate) * rect(t, pulse_duration, delay1)
chirp2 = chirp_signal(t - delay2, chirp_rate) * rect(t, pulse_duration, delay2)

# Summation => Simulated received signal
rx_signal = chirp1 + chirp2

# Add noise based on SNR
signal_power = np.mean(np.abs(rx_signal)**2)  # Compute signal power
noise_power = signal_power / (10**(SNR_dB / 10))  # Convert SNR dB to linear scale
noise = np.sqrt(noise_power) * (np.random.randn(len(t)) + 1j * np.random.randn(len(t)))  # Noise signal (for both I and Q)
rx_signal_noisy = rx_signal + noise  # Noisy received signal

# Compute cross-correlation
num_shifts = 2 * int(total_time * nyquist_frequency)
shifts = np.linspace(0, total_time, num_shifts)

# Convert time shifts to range shifts
range_shifts = (c * shifts) / 2

corr_values = np.zeros(num_shifts, dtype=complex)

# Define a reference chirp with no delay
ref_signal = chirp_signal(t, chirp_rate)

# Compute cross-correlation for each shift
for i, shift in enumerate(shifts):
    shifted_ref = chirp_signal(t - shift, chirp_rate) * rect(t, pulse_duration, shift)
    overlap = rx_signal_noisy * np.conjugate(shifted_ref)
    corr_values[i] = np.sum(overlap)

# Plot results
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(8, 10), sharex=False)
plt.tight_layout(pad=3.0)

# Individual delayed chirps
ax0.set_title('Individual Target Echoes')
ax0.plot(range_axis, np.real(chirp1), color='blue', label=f'Target 1 (Range={target_range1} m)')
ax0.plot(range_axis, np.real(chirp2), color='red', label=f'Target 2 (Range={target_range2} m)')
ax1.set_xlim(range_axis[0], range_axis[-1])
ax0.set_ylim(-1.2, 1.2)
ax0.legend()
ax0.set_xlabel('Range (m)')
ax0.set_ylabel('Amplitude')

# Received noisy signal
ax1.set_title(f'Received Signal with Noise (SNR={SNR_dB} dB)')
ax1.plot(range_axis, np.real(rx_signal_noisy), color='blue', label='Noisy Received Signal')
ax1.axvline(delay1, color='red', linestyle='--', label=f'Target 1 = {target_range1} m')
ax1.axvline(delay2, color='green', linestyle='--', label=f'Target 2 = {target_range2} m')
ax1.legend()
ax1.set_xlabel('Range (m)')
ax1.set_ylabel('Amplitude')
ax1.set_xlim(range_axis[0], range_axis[-1])

# Cross-correlation output
ax2.set_title('Cross-Correlation Output (Magnitude) in Range')
ax2.axvline(target_range1, color='red', linestyle='--', label=f'Target 1 = {target_range1} m')
ax2.axvline(target_range2, color='green', linestyle='--', label=f'Target 2 = {target_range2} m')
ax2.plot(range_shifts, np.abs(corr_values), 'g', label='|Correlation|')
ax2.legend()
ax2.set_xlabel('Range (m)')
ax2.set_ylabel('Magnitude')
ax1.set_xlim(range_axis[0], range_axis[-1])
ax2.set_ylim(0, 1.1 * np.max(np.abs(corr_values)))

plt.show()